In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import itertools
import numpy as np
from sklearn import preprocessing as pre
from opinions import Activation, OpinionatedAgent, Visualize

np.random.seed(0)
NAGENTS = 100
MOMENTUM_GAIN = 0.1


In [20]:
# Routines for visualizing the simulations 
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Spectral11
from bokeh.io import output_notebook

output_notebook()

def plot_agents(agents):
    p = figure(plot_width=800, plot_height=250)

    for agent, color in zip(agents, itertools.cycle(Spectral11)):
        p.line(x=np.arange(len(agent.opinion_history)),
               y=agent.opinion_history,
               color=color)

    show(p)

Loading BokehJS ...

In [21]:

def run_simulation(agents, interaction, iterations=100):


    # Simulation drivers 
    def find_update(agent, agents):
        opinion_vect = np.full((len(agents),), agent.opinion)
        other_opinions = [a.opinion for a in agents]
        delta_opinions = interaction(opinion_vect, other_opinions)

        weights = adj_matrix[agent.unique_id,:]
        update = np.dot(delta_opinions, weights)
        return update

    def iteration(i):
        # Find update values first, then apply updates - so don't incorporate updates in calculations of same iteration
        updates = [find_update(a, agents) for a in agents]
        for agent, update in zip(agents, updates):
            agent.update_opinion(update)

    for i in range(iterations):
        iteration(i) 


In [22]:
# Build a random adjacency matrix
random_matrix = np.random.rand(NAGENTS,NAGENTS)
symmetric_matrix = np.dot(random_matrix, random_matrix.T)
np.fill_diagonal(symmetric_matrix, 0)
normalized = pre.normalize(symmetric_matrix)

adj_matrix = normalized

In [23]:
# Simple simulation, no momentum
interaction = Activation(learning_rate=0.05)
agents = [OpinionatedAgent(i, "", momentum_gain=0.0) for i in range(NAGENTS)]
run_simulation(agents, interaction, 30)
plot_agents(agents)

In [24]:
interaction = Activation(learning_rate=0.01)
agents = [OpinionatedAgent(i, "", momentum_gain=0.2) for i in range(NAGENTS)]
run_simulation(agents, interaction)
plot_agents(agents)

In [25]:
interaction = Activation(learning_rate=0.01)
agents = [OpinionatedAgent(i, "", momentum_gain=0.5) for i in range(NAGENTS)]
run_simulation(agents, interaction)
plot_agents(agents)